In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from ISLP import load_data

# Prevent warnings 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#### Q8
This problem involves the OJ data set which is part of the ISLP package.

In [2]:
OJ = load_data('OJ')
OJ['Store7'] = OJ['Store7'].replace({'Yes':1,'No':0}).astype(int) 
OJ.head()

,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,0,0.000000,0.000000,0.24,1
1,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,0,0.150754,0.000000,0.24,1
2,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,0,0.000000,0.091398,0.23,1
3,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,0,0.000000,0.000000,0.00,1
4,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,1,0.000000,0.000000,0.00,0


(a) Create a training set containing a random sample of 800 observations, and a test set containing the remaining observations.

In [3]:
X = OJ[OJ.columns.drop('Purchase')]
y = OJ['Purchase']

X = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=800, random_state=0)

(b) Fit a support vector classifier to the training data using C = 0.01, with Purchase as the response and the other variables as predictors. How many support points are there?

In [4]:
linear_model = SVC(C=0.01, kernel='linear', random_state=0)
linear_model.fit(X_train, y_train)

n_support_vectors = len(linear_model.support_) # Indices of support vectors 
print(f"There are {n_support_vectors} support vectors")

There are 429 support vectors


(c) What are the training and test error rates?

In [5]:
tr_pred = linear_model.predict(X_train)
te_pred = linear_model.predict(X_test)
    
tr_acc = accuracy_score(tr_pred, y_train)
te_acc = accuracy_score(te_pred, y_test)
    
print(f"Training accuracy is {np.round(tr_acc,2)}")
print(f"Testing accuracy is {np.round(te_acc,2)}")

Training accuracy is 0.83
Testing accuracy is 0.81


(d) Use cross-validation to select an optimal C. Consider values in the range 0.01 to 10.

In [6]:
def tune_model(k, **kwargs):
    c_values = np.linspace(0.01,10,50)
    results = pd.DataFrame(columns=['C', 'Acc'])
    for c_i in c_values:
        model = SVC(C=c_i, kernel=k, random_state=0, **kwargs)
        scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')
        acc = np.mean(scores)
        new_row = pd.DataFrame({'C': [c_i], 'Acc': [acc]})
        results = pd.concat([results, new_row], ignore_index=True)
    return results

results_linear = tune_model('linear')

max_acc_row = results_linear.loc[results_linear['Acc'].idxmax()]
best_C_l = max_acc_row['C']

print(f"The optimal value of C is {best_C_l}")

The optimal value of C is 6.1263265306122445


(e) Compute the training and test error rates using this new value for C

In [7]:
best_linear = SVC(C=best_C_l, kernel='linear', random_state=0).fit(X_train, y_train)

pred_tr = best_linear.predict(X_train)
pred_te = best_linear.predict(X_test)
tr_acc_lin = np.round(accuracy_score(pred_tr, y_train),2)
te_acc_lin = np.round(accuracy_score(pred_te, y_test),2)

print(f"Best training accuracy is {tr_acc_lin}")
print(f"Best testing accuracy is {te_acc_lin}")

Best training accuracy is 0.84
Best testing accuracy is 0.81


(f) Repeat parts (b) through (e) using a support vector machine with a radial kernel. Use the default value for gamma.

In [8]:
results_radial = tune_model('rbf')

max_acc_row = results_radial.loc[results_radial['Acc'].idxmax()]
best_C_r = max_acc_row['C']

print(f"The optimal value of C is {best_C_r}")

best_radial = SVC(C=best_C_r, kernel='rbf', random_state=0).fit(X_train, y_train)

pred_tr = best_radial.predict(X_train)
pred_te = best_radial.predict(X_test)
tr_acc_rad = np.round(accuracy_score(pred_tr, y_train),2)
te_acc_rad = np.round(accuracy_score(pred_te, y_test),2)

print(f"Best training accuracy is {tr_acc_rad}")
print(f"Best testing accuracy is {te_acc_rad}")

The optimal value of C is 0.8255102040816327
Best training accuracy is 0.85
Best testing accuracy is 0.82


(g) Repeat parts (b) through (e) using a support vector machine with a polynomial kernel. Set degree = 2.

In [9]:
results_poly = tune_model('poly', degree=2)

max_acc_row = results_poly.loc[results_poly['Acc'].idxmax()]
best_C_p = max_acc_row['C']

print(f"The optimal value of C is {best_C_p}")

best_poly = SVC(C=best_C_p, kernel='poly', degree=2, random_state=0).fit(X_train, y_train)

pred_tr = best_poly.predict(X_train)
pred_te = best_poly.predict(X_test)
tr_acc_poly = np.round(accuracy_score(pred_tr, y_train),2)
te_acc_poly = np.round(accuracy_score(pred_te, y_test),2)

print(f"Best training accuracy is {tr_acc_poly}")
print(f"Best testing accuracy is {te_acc_poly}")

The optimal value of C is 9.796122448979592
Best training accuracy is 0.79
Best testing accuracy is 0.76


(h) Overall, which approach seems to give the best results on this data?

>- The radial kernel seems to give the best results on this data.
>- It's surprising to note that the linear kernel performs better than the polynomial kernel, given that the true boundary may be non-linear. 